Libraries and Imports

In [1]:
import json
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.utils.data as data
import torchmetrics
from ncps.torch import LTC
from ncps.wirings import AutoNCP
from plyer import notification
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger

sys.path.append(os.path.abspath("funcs"))

from config_reading import read_and_validate_json
from latest_log import get_latest_csv
from timer_callback import TimingCallback

Opening configuration file

In [2]:
config = read_and_validate_json("config.json")

Setting the CUDA float32 precision.

In [3]:
torch.set_float32_matmul_precision(config["float_precision"])

Set the seed manually to ensure reproducibility

In [4]:
torch.manual_seed(config["seed"])
torch.cuda.manual_seed(config["seed"])
torch.cuda.manual_seed_all(config["seed"])
np.random.seed(config["seed"])
pl.seed_everything(config["seed"], workers=True)

Seed set to 115


115

Reading the dataset.

In [5]:
train_x = torch.load(os.path.join(config["save_path"], "tensor_train_x.pt"))
train_y = torch.load(os.path.join(config["save_path"], "tensor_train_y.pt"))
val_x = torch.load(os.path.join(config["save_path"], "tensor_val_x.pt"))
val_y = torch.load(os.path.join(config["save_path"], "tensor_val_y.pt"))
test_x = torch.load(os.path.join(config["save_path"], "tensor_test_x.pt"))
test_y = torch.load(os.path.join(config["save_path"], "tensor_test_y.pt"))

In [6]:
has_validation = config["validation_proportion"] > 0

Defining loaders, models phases and model configuration

In [7]:
train_x.shape[0]

6616

In [8]:
test_x.shape[0]

2947

In [9]:
val_x.shape[0]

736

In [ ]:
if config["batch_size"] == "full":
  config["batch_size"] = 6616

In [ ]:
train_dataloader = data.DataLoader(data.TensorDataset(train_x, train_y), batch_size=6616, shuffle=True, num_workers=16, persistent_workers=True)
if has_validation:
	val_dataloader = data.DataLoader(data.TensorDataset(val_x, val_y), batch_size=735, num_workers=16, persistent_workers=True)
test_dataloader = data.DataLoader(data.TensorDataset(test_x, test_y), batch_size=2947, num_workers=16, persistent_workers=True)

In [ ]:
class SequenceLearner(pl.LightningModule):
	def __init__(self, model, lr):
		super().__init__()
		self.model = model
		self.lr = lr
		self.loss_fn = nn.CrossEntropyLoss()
		self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=6)
		self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=6)
		self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=6)
		self.log_collection = []

	def training_step(self, batch, batch_idx):
		x, y = batch
		y_hat, _ = self.model.forward(x)
		loss = self.loss_fn(y_hat, y)
		y_pred = y_hat.argmax(dim=-1)
		self.log_collection.append({"loss": loss, "acc": self.train_acc(y_pred, y)})
		return {"loss": loss}

	def validation_step(self, batch, batch_idx):
		x, y = batch
		y_hat, _ = self.model.forward(x)
		loss = self.loss_fn(y_hat, y)
		y_pred = y_hat.argmax(dim=-1)
		self.log_collection.append({"loss": loss, "acc": self.val_acc(y_pred, y)})
		return {"loss": loss}
	
	def test_step(self, batch, batch_idx):
		x, y = batch
		y_hat, _ = self.model.forward(x)
		loss = self.loss_fn(y_hat, y)
		y_pred = y_hat.argmax(dim=-1)
		self.log("test_loss", loss)
		self.log("test_acc", self.test_acc(y_pred, y))
		return {"loss": loss}

	def on_validation_epoch_end(self):
		self.log("train_loss", self.log_collection[0]["loss"], on_step=False, on_epoch=True)
		self.log("train_acc", self.log_collection[0]["acc"], on_step=False, on_epoch=True)
		self.log("val_loss", self.log_collection[1]["loss"], on_step=False, on_epoch=True)
		self.log("val_acc", self.log_collection[1]["acc"], on_step=False, on_epoch=True)
		self.log_collection = []


	def configure_optimizers(self):
		return torch.optim.Adam(self.model.parameters(), lr=self.lr)

In [ ]:
out_features = 6 # Output
in_features = 561 # Input

In [ ]:
wiring = AutoNCP(config["num_neurons"], out_features)

In [ ]:
ltc_model = LTC(in_features, wiring, batch_first=True)
learn = SequenceLearner(ltc_model, lr=config["learning_rate"])

log_dir = f"logs"
logger = CSVLogger(log_dir, name=config["model_name"])

checkpoint_dir = f"{log_dir}/{config["model_name"]}/checkpoints"
last_checkpoint_path = f"{checkpoint_dir}/last.ckpt"
checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoint_dir,
    filename="{epoch}-{val_loss:.2f}",
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    save_last=True
)

trainer = pl.Trainer(
	logger=logger,
	max_epochs=config["max_epochs"],
  check_val_every_n_epoch=1,
  callbacks=[TimingCallback(), checkpoint_callback],
	gradient_clip_val=1  # Clip gradient to stabilize training
)

Training

In [16]:
if has_validation:
	trainer.fit(learn, train_dataloader, val_dataloader)
else:
	trainer.fit(learn, train_dataloader)

Testing and Ploting Results

In [ ]:
trainer.test(learn, test_dataloader)

In [ ]:
latest_csv = get_latest_csv(logger.log_dir)
metrics = pd.read_csv(latest_csv)

steps = metrics["epoch"]

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(steps, metrics["train_acc"], label='Train Accuracy')
if has_validation:
	plt.plot(steps, metrics["val_acc"], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
latest_csv = get_latest_csv(logger.log_dir)
metrics = pd.read_csv(latest_csv)

steps = metrics["epoch"]

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(steps, metrics["train_loss"], label='Train Loss')
if has_validation:
	plt.plot(steps, metrics["val_loss"], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

Saving the trained model

In [ ]:
if not os.path.exists("models"):
    os.makedirs("models")

torch.save(ltc_model, f"models/{config["model_name"]}.pt")

Saving the configuration used

In [ ]:
config_file_path = os.path.join(logger.log_dir, "config.json")
with open(config_file_path, 'w') as config_file:
	json.dump(config, config_file, indent=2)

Notification for end of training

In [ ]:
notification.notify(
	title="Training ended",
	message=f"The training of the model {config["model_name"]} with {config["max_epochs"]} epochs has been completed.",
	timeout=10
)